In [26]:
import pandas as pd
import numpy as np
import sklearn as sk
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
import pickle
from sklearn.tree import export_graphviz
from sklearn import tree
from dtreeviz.trees import dtreeviz
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import streamlit as st

In [27]:
train = pd.read_csv('train.csv')

train = train.dropna()

train['TotalApplicantIncome'] = train['ApplicantIncome'] + train['ApplicantIncome']

gender_dummies = pd.get_dummies(train['Gender'])
train = pd.concat((train, gender_dummies), axis = 1)
train = train.drop(['Gender'], axis = 1)
train = train.drop(['Male'], axis = 1)
train = train.rename(columns = {'Female' : 'Gender'})

married_dummies = pd.get_dummies(train['Married'])
train = pd.concat((train, married_dummies), axis = 1)
train = train.drop(['Married'], axis = 1)
train = train.drop(['No'], axis = 1)
train = train.rename(columns = {'Yes' : 'Married'})

LoanStatus_dummies = pd.get_dummies(train['Loan_Status'])
train = pd.concat((train, LoanStatus_dummies), axis = 1)
train = train.drop(['Loan_Status'], axis = 1)
train = train.drop(['N'], axis = 1)
train = train.rename(columns = {'Y' : 'Loan_Approved'})

train.astype({'Credit_History' : int})

,Loan_ID,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,TotalApplicantIncome,Gender,Married,Loan_Approved
1,LP001003,1,Graduate,No,4583,1508.0,128.0,360.0,1,Rural,9166,0,1,0
2,LP001005,0,Graduate,Yes,3000,0.0,66.0,360.0,1,Urban,6000,0,1,1
3,LP001006,0,Not Graduate,No,2583,2358.0,120.0,360.0,1,Urban,5166,0,1,1
4,LP001008,0,Graduate,No,6000,0.0,141.0,360.0,1,Urban,12000,0,0,1
5,LP001011,2,Graduate,Yes,5417,4196.0,267.0,360.0,1,Urban,10834,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,Graduate,No,2900,0.0,71.0,360.0,1,Rural,5800,1,0,1
610,LP002979,3+,Graduate,No,4106,0.0,40.0,180.0,1,Rural,8212,0,1,1
611,LP002983,1,Graduate,No,8072,240.0,253.0,360.0,1,Urban,16144,0,1,1
612,LP002984,2,Graduate,No,7583,0.0,187.0,360.0,1,Urban,15166,0,1,1


In [28]:
%%writefile app.py
 
#import pickle
#import streamlit as st
 
# loading the trained model
#pickle_in = open('classifier.pkl', 'rb') 
#classifier = pickle.load(pickle_in)

@st.cache()

#independent variables
features = ['Gender', 'Married', 'TotalApplicantIncome', 'LoanAmount', 'Credit_History']
X = train[features]
#dependent variables
dependent = 'Loan_Approved'
y = train[dependent]

#split dataset into train (80%) and test (20%), shuffle observations
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 10, shuffle = True)
    
model = RandomForestClassifier(max_depth=4, random_state = 10, n_estimators = 100) 
model.fit(x_train, y_train)
    
    
  
# defining the function which will make the prediction using the data which the user inputs 
def prediction(Gender, Married, TotalApplicantIncome, LoanAmount, Credit_History):   
    
    if Gender == "Male":
        Gender = 0
    else:
        Gender = 1
 
    if Married == "Unmarried":
        Married = 0
    else:
        Married = 1
 
    if Credit_History == "No Credit History":
        Credit_History = 0
    else:
        Credit_History = 1  
 
    LoanAmount = LoanAmount / 1000
 
    # Making predictions 
    pred_inputs = model.predict([[Gender, Married, TotalApplicantIncome, LoanAmount, Credit_History]])
        
        
    if prediction == 0:
        pred = 'I am sorry, you have been rejected for the loan.'
    else:
        pred = 'Congrats! You have been approved for the loan!'
    return pred
      
  
# this is the main function in which we define our webpage  
def main():       
    # front end elements of the web page 
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Streamlit Loan Prediction ML App</h1> 
    </div> 
    """
      
    # display the front end aspect
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # following lines create boxes in which user can enter data required to make prediction 
    Gender = st.selectbox('Gender',("Male","Female"))
    Married = st.selectbox('Marital Status',("Unmarried","Married")) 
    ApplicantIncome = st.number_input("Total Monthly Income, (Include Coborrower if Applicable)") 
    LoanAmount = st.number_input("Loan Amount")
    Credit_History = st.selectbox('Credit History',("Has Credit History","No Credit History"))
    result =""
      
    # when 'Predict' is clicked, make the prediction and store it 
    if st.button("Predict"): 
        result = prediction(Gender, Married, ApplicantIncome, LoanAmount, Credit_History) 
        st.success('Final Decision: {}'.format(result))
        print(LoanAmount)
     
if __name__=='__main__': 
    main()

Overwriting app.py


In [24]:

# #original

# %%writefile app.py
 
# import pickle
# import streamlit as st
 
# # loading the trained model
# pickle_in = open('classifier.pkl', 'rb') 
# classifier = pickle.load(pickle_in)
 
# @st.cache()
  
# # defining the function which will make the prediction using the data which the user inputs 
# def prediction(Gender, Married, ApplicantIncome, LoanAmount, Credit_History):   
 
#     # Pre-processing user input    
#     if Gender == "Male":
#         Gender = 0
#     else:
#         Gender = 1
 
#     if Married == "Unmarried":
#         Married = 0
#     else:
#         Married = 1
 
#     if Credit_History == "Unclear Debts":
#         Credit_History = 0
#     else:
#         Credit_History = 1  
 
#     LoanAmount = LoanAmount / 1000
 
#     # Making predictions 
#     prediction = classifier.predict( 
#         [[Gender, Married, ApplicantIncome, LoanAmount, Credit_History]])
     
#     if prediction == 0:
#         pred = 'Rejected'
#     else:
#         pred = 'Approved'
#     return pred
      
  
# # this is the main function in which we define our webpage  
# def main():       
#     # front end elements of the web page 
#     html_temp = """ 
#     <div style ="background-color:yellow;padding:13px"> 
#     <h1 style ="color:black;text-align:center;">Streamlit Loan Prediction ML App</h1> 
#     </div> 
#     """
      
#     # display the front end aspect
#     st.markdown(html_temp, unsafe_allow_html = True) 
      
#     # following lines create boxes in which user can enter data required to make prediction 
#     Gender = st.selectbox('Gender',("Male","Female"))
#     Married = st.selectbox('Marital Status',("Unmarried","Married")) 
#     ApplicantIncome = st.number_input("Applicants monthly income") 
#     LoanAmount = st.number_input("Total loan amount")
#     Credit_History = st.selectbox('Credit_History',("Unclear Debts","No Unclear Debts"))
#     result =""
      
#     # when 'Predict' is clicked, make the prediction and store it 
#     if st.button("Predict"): 
#         result = prediction(Gender, Married, ApplicantIncome, LoanAmount, Credit_History) 
#         st.success('Your loan is {}'.format(result))
#         print(LoanAmount)
     
# if __name__=='__main__': 
#     main()
    


SyntaxError: invalid syntax (<ipython-input-24-d538a19b1393>, line 76)